In [1]:
import org.apache.spark.sql.functions._

In [2]:
val LOC = "/home/ubuntu/data/"
val FILE = "Egypt.csv"

print(LOC+FILE)

/home/ubuntu/data/Egypt.csv

LOC = /home/ubuntu/data/
FILE = Egypt.csv


Egypt.csv

In [3]:
// Read the file
val ds = spark.read.format("csv")
         .option("header", "false")   // Keep the headers: "true" skips them
         .option("mode", "DROPMALFORMED")
         .load(LOC+FILE)

ds = [_c0: string, _c1: string ... 4 more fields]


[_c0: string, _c1: string ... 4 more fields]

In [3]:
print(ds.getClass)    // get object type

class org.apache.spark.sql.Dataset

In [4]:
ds.show(5)      // equivalent to "head"

+-----+--------------------+------------+--------------------+----------+----------------+
|  _c0|                 _c1|         _c2|                 _c3|       _c4|             _c5|
+-----+--------------------+------------+--------------------+----------+----------------+
|Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-01-01|284533.954567226|
|Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-02-01|263907.051428571|
|Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-03-01|351187.960412742|
|Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-04-01|308846.532766667|
|Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-05-01| 258098.75879821|
+-----+--------------------+------------+--------------------+----------+----------------+
only showing top 5 rows



In [5]:
print(ds.first())      // Get the first row

[Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-01-01,284533.954567226]

In [6]:
// rename ALL columns
val newNames = Seq("country", "product", "account","type", "month", "amount")
val ds2 = ds.toDF(newNames: _*)

newNames = List(country, product, account, type, month, amount)
ds2 = [country: string, product: string ... 4 more fields]


[country: string, product: string ... 4 more fields]

In [19]:
ds2.printSchema        // column names and types

root
 |-- country: string (nullable = true)
 |-- product: string (nullable = true)
 |-- account: string (nullable = true)
 |-- type: string (nullable = true)
 |-- month: string (nullable = true)
 |-- amount: string (nullable = true)



In [20]:
ds.describe().show()   // stats on the columns

+-------+-----+--------------------+------------+--------------------+----------+--------------------+
|summary|  _c0|                 _c1|         _c2|                 _c3|       _c4|                 _c5|
+-------+-----+--------------------+------------+--------------------+----------+--------------------+
|  count|45901|               45901|       45901|               45901|     45901|               45901|
|   mean| null|                null|        null|                null|      null|   5308318.084986666|
| stddev| null|                null|        null|                null|      null|2.2177794594817642E7|
|    min|Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-01-01|-0.00000000069849...|
|    max|Egypt|Small Business Pr...|GG0000008024|Processed-Issuing...|2018-05-01|        9999.9780935|
+-------+-----+--------------------+------------+--------------------+----------+--------------------+



In [38]:
val colCount = ds.columns.length             // get number of columns
println("file has " + colCount + " columns")

val rowCount = ds.count                          // get number of rows
println("file has " + rowCount + " rows")

val formatter = java.text.NumberFormat.getIntegerInstance        // row count with comma
print("file has " +  formatter.format(ds.count())+" rows")

file has 6 columns
file has 45901 rows
file has 45,901 rows

colCount = 6
rowCount = 45901
formatter = java.text.DecimalFormat@674dc


java.text.DecimalFormat@674dc

In [42]:
for(a <- ds.columns) println(a)        // print Column names

_c0
_c1
_c2
_c3
_c4
_c5


In [44]:
// Iterate over some rows
val first4 = ds.take(4)
for (x <- first4) println(x)

[Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-01-01,284533.954567226]
[Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-02-01,263907.051428571]
[Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-03-01,351187.960412742]
[Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-04-01,308846.532766667]


first4 = Array([Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-01-01,284533.954567226], [Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-02-01,263907.051428571], [Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-03-01,351187.960412742], [Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-04-01,308846.532766667])


Array([Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-01-01,284533.954567226], [Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-02-01,263907.051428571], [Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-03-01,351187.960412742], [Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-04-01,308846.532766667])

In [45]:
val ds3 = ds2.withColumnRenamed("account", "customer")   // rename a SINGLE column
ds3.printSchema

root
 |-- country: string (nullable = true)
 |-- product: string (nullable = true)
 |-- customer: string (nullable = true)
 |-- type: string (nullable = true)
 |-- month: string (nullable = true)
 |-- amount: string (nullable = true)



ds3 = [country: string, product: string ... 4 more fields]


[country: string, product: string ... 4 more fields]

In [46]:
// Add a column
import org.apache.spark.sql.functions.lit    // needed if you're adding the same value for each row
val newDF = ds3.withColumn("new", lit(10))    // adding value 10 to all records

newDF.show(5)

+-------+--------------------+------------+--------------------+----------+----------------+---+
|country|             product|    customer|                type|     month|          amount|new|
+-------+--------------------+------------+--------------------+----------+----------------+---+
|  Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-01-01|284533.954567226| 10|
|  Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-02-01|263907.051428571| 10|
|  Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-03-01|351187.960412742| 10|
|  Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-04-01|308846.532766667| 10|
|  Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-05-01| 258098.75879821| 10|
+-------+--------------------+------------+--------------------+----------+----------------+---+
only showing top 5 rows



newDF = [country: string, product: string ... 5 more fields]


[country: string, product: string ... 5 more fields]

In [47]:
// Filter a numeric
val before = newDF.count()
val filtered = newDF.filter($"amount" > 500000.0)
val after = filtered.count()

println("Removed " + formatter.format( before-after) + " rows. " + formatter.format(after) +" remaining")

Removed 23,877 rows. 22,024 remaining


before = 45901
filtered = [country: string, product: string ... 5 more fields]
after = 22024


22024

### Nulls

In [54]:
ds2.filter("amount is null").show      // check if a column has Nulls

+-------+-------+-------+----+-----+------+
|country|product|account|type|month|amount|
+-------+-------+-------+----+-----+------+
+-------+-------+-------+----+-----+------+



In [59]:
println(ds2.filter($"amount".isNull).count)    // count the number of nulls

0


In [58]:
print(ds2.filter("amount is not null").count)    // count the number of nulls

45901

In [61]:
case class MyRow(x: Double, y:String)     // creating data
val data = Seq(MyRow(1.1, "bat"),MyRow(2.2, "cat"),MyRow(Double.NaN, "dog"))

defined class MyRow
data = List(MyRow(1.1,bat), MyRow(2.2,cat), MyRow(NaN,dog))


List(MyRow(1.1,bat), MyRow(2.2,cat), MyRow(NaN,dog))

In [62]:
val df = data.toDF("id", "typ")
df.show

+---+---+
| id|typ|
+---+---+
|1.1|bat|
|2.2|cat|
|NaN|dog|
+---+---+



df = [id: double, typ: string]


[id: double, typ: string]

In [63]:
// replaces Null with a static value
val t = df.na.fill(18)
t.show

+----+---+
|  id|typ|
+----+---+
| 1.1|bat|
| 2.2|cat|
|18.0|dog|
+----+---+



t = [id: double, typ: string]


[id: double, typ: string]

In [179]:
// equivalent to "dropna"
val t = df.na.drop()
t.show

+---+---+
| id|typ|
+---+---+
|1.1|bat|
|2.2|cat|
+---+---+



t = [id: double, typ: string]


[id: double, typ: string]

In [5]:
val col = Array("id")

col = Array(id)


Array(id)

In [6]:
import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer()
  .setInputCols(col)
  .setOutputCols(col.map(c => s"${c}_imputed")
  .setStrategy("mean")

Name: Syntax Error.
Message: 
StackTrace: 

In [183]:
imputer.fit(df).transform(df)

Name: java.lang.IllegalArgumentException
Message: requirement failed: Column typ must be of type equal to one of the following types: [double, float] but was actually of type string.
StackTrace:   at scala.Predef$.require(Predef.scala:224)
  at org.apache.spark.ml.util.SchemaUtils$.checkColumnTypes(SchemaUtils.scala:60)
  at org.apache.spark.ml.feature.ImputerParams$$anonfun$2.apply(Imputer.scala:76)
  at org.apache.spark.ml.feature.ImputerParams$$anonfun$2.apply(Imputer.scala:74)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
  at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
  at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
  at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
  at scala.collection.mutable.ArrayOps$ofRef.map(ArrayOps.scala:186)
  at org.apache.spark.ml.feature.Impute

### GroupBy

In [132]:
ds2.columns

Array(country, product, account, type, month, amount)

In [133]:
val g = ds2.groupBy("product")

g = RelationalGroupedDataset: [grouping expressions: [product: string], value: [country: string, product: string ... 4 more fields], type: GroupBy]


RelationalGroupedDataset: [grouping expressions: [product: string], value: [country: string, product: string ... 4 more fields], type: GroupBy]

In [203]:
g.count.orderBy("count").show(5)    // sort the groups by Count

+--------------------+-----+
|             product|count|
+--------------------+-----+
|MasterCard World ...|  634|
|MasterCard World ...|  813|
|MasterCard Debit ...|  899|
|MasterCard Gold D...|  917|
|MasterCard Commer...|  968|
+--------------------+-----+
only showing top 5 rows



In [202]:
g.count().orderBy($"count".desc).show(5)    // Count descending

+--------------------+-----+
|             product|count|
+--------------------+-----+
|MasterCard Standa...| 5357|
|Mastercard Platin...| 4700|
|MasterCard Gold C...| 4617|
|MasterCard Standa...| 4281|
|         All Prepaid| 3969|
+--------------------+-----+
only showing top 5 rows



In [201]:
g.agg(min("amount")).show(5)    // get the min for each group

+--------------------+-------------+
|             product|  min(amount)|
+--------------------+-------------+
|MasterCard World ...|-50.145084516|
|Maestro Consumer ...|            0|
|MasterCard Titani...|   0.74104271|
|Small Business Pr...|            0|
|         All Prepaid|    10.022464|
+--------------------+-------------+
only showing top 5 rows



### Working with Columns

In [209]:
val prod = ds2("product")

prod = product


product

In [221]:
ds2.select("product").show(5)    // view a column

+--------------------+
|             product|
+--------------------+
|All Cirrus Issuin...|
|All Cirrus Issuin...|
|All Cirrus Issuin...|
|All Cirrus Issuin...|
|All Cirrus Issuin...|
+--------------------+
only showing top 5 rows



In [233]:
ds2.select(ds2("country")).distinct.show    // Get unique values

+-------+
|country|
+-------+
|  Egypt|
+-------+



In [223]:
ds2.select(ds2("amount").cast("float")).show(5)    // cast column type

+---------+
|   amount|
+---------+
|284533.97|
|263907.06|
|351187.97|
|308846.53|
|258098.77|
+---------+
only showing top 5 rows



In [226]:
ds2.orderBy($"amount".desc).show(5)   // sort dataset by Column

+-------+--------------------+------------+--------------------+----------+--------------+
|country|             product|     account|                type|     month|        amount|
+-------+--------------------+------------+--------------------+----------+--------------+
|  Egypt|Small Business Pr...|GG0000006914|Processed-Acquiri...|2016-09-01|  9999.9780935|
|  Egypt|MasterCard Titani...|GG0000005125|Processed-Gross D...|2016-06-01|  9999.9780935|
|  Egypt|Small Business Pr...|GG0000001090|Processed-Gross D...|2016-10-01|9999.945399403|
|  Egypt|MasterCard World ...|GG0000004127|Processed-Acquiri...|2017-05-01|9999.853608613|
|  Egypt|MasterCard World ...|GG0000004127|Processed-Acquiri...|2018-01-01|9999.491111226|
+-------+--------------------+------------+--------------------+----------+--------------+
only showing top 5 rows



In [19]:
df.reduce {
    (x, y) =>
    if (x.getAs[Int]("timestamp") > y.getAs[Int]("timestamp")) x else y
}

23676